In [13]:
from __future__ import print_function, division
import cv2
import torch
import numpy as np
import pickle
import math

In [14]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
data = pickle.load(open("toy_example/data/toyexample_1_data.p", 'rb'))
img = cv2.imread("toy_example/data/toyexample_1.png", 0)

In [36]:
T_perturbed = np.array([[math.cos(5*math.pi/180), -math.sin(5*math.pi/180), 0, 0],
             [math.sin(5*math.pi/180), math.cos(5*math.pi/180), 0, 0],
             [0, 0, 1, 0]])

In [37]:
data['coords'] = np.around(data['2d_points']).astype(int) - 1

In [38]:
P_perturbed = np.dot(data['K'], T_perturbed)
projected_2d = np.dot(P_perturbed, np.concatenate((data['3d_points'], np.ones(len(data['3d_points']))[:, None]),-1).T)
projected_2d = (projected_2d.T/projected_2d.T[:,2,None])[:, :2]

In [39]:
coords_2d = np.around(projected_2d)
coords_2d = coords_2d.astype(int) - 1

In [40]:
img = img.astype('uint8')
for i, p in enumerate(data['coords']):
    print(i, p)
    cv2.circle(img, tuple(p), 1, (128, 128, 0), -1)

0 [ 59 104]
1 [86 72]
2 [ 34 110]
3 [42 73]
4 [78 48]
5 [117  42]
6 [87 87]
7 [64 85]
8 [ 80 109]
9 [56 54]
10 [73 98]
11 [102 112]
12 [96 45]
13 [ 78 118]


In [41]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### run BA

In [42]:
%load_ext autoreload
%autoreload 2

from featureBA.src.model import sparse3DBA
from featureBA.src.utils import sobel_filter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
img = cv2.imread("toy_example/data/toyexample_1.png", 0)
img = img.astype('double')

In [44]:
img_torch = torch.from_numpy(img)[None,...]
grad_x, grad_y = sobel_filter(img_torch)

In [46]:
cv2.imshow('image', grad_x.numpy().reshape(img.shape))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
pts3D = torch.from_numpy(np.delete(data['3d_points'], 6, 0)[:,:3])
ref2d = torch.from_numpy(np.flip(data['coords']).copy())
feature_ref = torch.cat([img_torch[:, i, j].unsqueeze(0) for i, j in zip(ref2d[:,0], ref2d[:,1])]).type(torch.DoubleTensor)
feature_map_query = img_torch.type(torch.DoubleTensor)
R_init, t_init = torch.from_numpy(T_perturbed[:, :3]), torch.from_numpy(T_perturbed[:, 3])
feature_grad_x = grad_x
feature_grad_y = grad_y

In [48]:
s3dba = sparse3DBA(100)

In [49]:
R, t = s3dba(pts3D, feature_ref, feature_map_query, grad_x, grad_y, data['K'], R_init, t_init)

RuntimeError: The size of tensor a (13) must match the size of tensor b (14) at non-singleton dimension 0

In [50]:
data['2d_points'].shape

(14, 2)

In [ ]:
data['']